# Tutorial for Robotic Modeling using Mujoco 

This tutorial covers the basics of the physics modeling with Mujoco. We will primarily focus on rigid-body systems that are prevalent in most robotic applications. 

## Principles of Physics Modeling
In order to simulate a physical system, we typically need to tell the simulator three things:
1. **System models**, often described as ODE/PDE or just functions. Often times, it is hard to derive analytically the system (differential) equations. Therefore, the most important functionality of a simulator is to allow the user to easily specify physical properties and automatically translate these properties into (differential) equations. For rigid body robotic systems, we need to tell the simulator:
   - **Muti-body system**: articulated rigid body system consists of multiple rigid bodies connected through joints. We need to define each body and describe its physical properties (e.g. pos, orientation, inertia matrix, etc), and how it is connected to adjacent bodies (through joints) 
   - **Environment/interaction**: gravity direction, ground, table, obstacle, wall, person, other robots, etc. How the robot interacts with the environments depends on many physical properties that can be modified, including friction, contact, collision mechanisms, 
   - **Actuators/sensors**: robot's motion is completely determined by each joint's motion. Joint's motion is determined by sensing and actuation strategies. The simulation is about the closed-loop behavior of the robot's motion and its interaction with the environment. 
2. **Physics Engine**: This part determines how to solve the (differential) equations associated with the specified robot-environment physical system. Parameters that can be modified include simulation time step, integrator (Euler, RK4, implicit, implicitfast), tolerance (to terminate iterative solver), constraint solver (PGS, CG, Newton), number of iterations (for constraint solver), etc. For Mujoco xml, these parameters are mostly specified through the \<option\> element. 
3. **Visualization**: This part determines how to visualize the robotic system scene. In Mujoco xml, some key visualization components are listed below:
   -  **light**: body/light will create a light, which moves with the body where it is defined. To create a fixed light, define it in the world body. The lights created here are in addition to the default headlight which is always defined and is adjusted via the visual element. The maximum number of lights that can be active simultaneously is 8, counting the headlight.
   -  **Camera**: body/⁠camera creates a camera, which moves with the body where it is defined. To create a fixed camera, define it in the world body. ca
   -  **geom**: body/⁠geom creates a geom, and attaches it rigidly to the body within which the geom is defined. Multiple geoms can be attached to the same body. At runtime they determine the appearance and collision properties of the body. 
   -  **Material**: asset/⁠material creates a material asset. It can be referenced from skins, geoms, sites and tendons to set their appearance. For rigid-body system, material is referenced from geom associated with a body. 
   - **Body vs Geom**: \<body\>: Represents a rigid entity with mass, inertia, and movable via joints. It does not have a shape until \<geom\> elements are attached to it. While \<geom\>: Defines the shape and size, used for collisions and visualization. It does not have mass or inertia, but can have density specified to infer mass. 



# Python Examples

## Example 1: Geometric objects

In [1]:
import numpy as np
import mujoco
import mujoco.viewer as viewer

xmlModel = """
<mujoco>
  <worldbody>
    <geom name="red_box" type="box" pos="0 0 0" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="blue_sphere" pos=".2 .2 .2" size=".1" rgba="0 0 1 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xmlModel)
data = mujoco.MjData(model)
viewer.launch(model,data)


The box and sphere do not move. They are geometric objects only for visulization. In order for them to move, they need to be attached to a body with motion freedom specified by a joint. Before adding joint, we first use asset to specify material and texture, use \<default\> to change the default setting for the geom element

In [2]:
xml_ground = """
<mujoco>
  <asset>  
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="6 6" texuniform="true" reflectance=".2"/>
    <material name="wall" rgba='.5 .5 .5 1'/>
  </asset>
  <default>
    <geom type="box" size=".05 .05 .05" />
  </default>

  <worldbody>
    <light name="light" pos="-.2 0 1"/>
    <geom name="ground" type="plane" size=".5 .5 10" material="grid" friction=".1"/>
    <camera name="y" pos="-.1 -.6 .3" xyaxes="1 0 0 0 1 2"/>
    <geom name="red_box" pos="0 0 0.2" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
    <geom name="blue_sphere" pos=".2 .2 .2" size=".1" rgba="0 0 1 1"/>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml_ground)
data = mujoco.MjData(model)
viewer.launch(model,data)

## Example 2： Multi-body Robotic Systems

Now let's add a body and a joint to the world frame (worldbody). Simulate the physical motion caused by gravity. 

In [5]:
xml = """
<mujoco>
  <option gravity="0 0 -9.81"/>        
  <worldbody>
  <geom name="ground" type="plane" pos="0 0 -1" size="10 10 10"/>
    <light name="top" pos="0 0 1"/>
    <body name="box_and_sphere" euler="0 0 -30">
      <joint name="swing" type="hinge" axis="0 1 0" pos="-.2 -.2 -.2"/>
      
      <geom name="red_box" type="box" size=".2 .2 .2" rgba="1 0 0 1"/>
      <geom name="green_sphere" pos=".2 .2 .2" size=".1" rgba="0 1 0 1"/>
    </body>
  </worldbody>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)
data = mujoco.MjData(model)
viewer.launch(model,data)

Now let's have a scene with ground (modeled as plane) and two boxes. The box can move freely in space, which is modeled as a "free" joint relative to the worldbody. You can modify the friction coefficients to see the resulting sliding motion. 

In [7]:
xml3 = """
<mujoco>
  <asset>  
    <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
     rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
    <material name="grid" texture="grid" texrepeat="6 6"
     texuniform="true" reflectance=".2"/>
     <material name="wall" rgba='.5 .5 .5 1'/>
  </asset>
  <default>
    <geom type="box" size=".05 .05 .05" />
    <joint type="free"/>
  </default>

  <worldbody>
    <light name="light" pos="-.2 0 1"/>
    <geom name="ground" type="plane" size="10 10 10" material="grid"
     zaxis="-.3 0 1" friction=".2"/>
    <camera name="y" pos="-.1 -.6 .3" xyaxes="1 0 0 0 1 2"/>
    <body pos="0 1 .3">
      <joint/>
      <geom friction="0.1"/>
    </body>
    <body pos="0 0 .3">
      <joint/>
      <geom friction="0.1"/>
    </body>
  </worldbody>

</mujoco>
"""

# load
model = mujoco.MjModel.from_xml_string(xml3)
data = mujoco.MjData(model)
viewer.launch(model,data)


Now, let's have a multi-body robot

To construct a multi-body robot, one needs to clearly specify
- body frame
- geom frame (the frame of geomoetric object)
- joint frame
For simplicity, we shall try to let the inertia frame coincide with the body frame. Example setup can be found in the following picture:


<img src="../assets/2RRobot.jpg" width="20%" alt="My Image">


In [8]:
import numpy as np
import mujoco
import mujoco.viewer as viewer

xml4="""<mujoco model="3R_robot">
    <compiler angle="degree"/>
    <asset>  
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
        rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
        <material name="grid" texture="grid" texrepeat="6 6" texuniform="true" reflectance=".2"/>
    </asset>
    <default>
        <joint type="hinge" axis="0 0 1" limited="true"/>
        <geom type="cylinder" size=".025 .1" />
    </default>

    <worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
        <geom type="plane" size="1 1 0.1" material="grid"/>        
        <body name="BaseLink" pos="0 0 0.1">  
            <geom type="cylinder" pos="0 0 0" size=".025 .1" />  
            <body name="link1" pos="0 0.1 0.125" euler="-90 0 0">
                <joint name="joint1" pos="0 0 -0.1" range="-90 90" axis ="0 1 0"/>
                <geom pos="0 0 0" rgba=".6 .2 .2 1"/>
                <site name="torque_site" pos="0 0.2 0"/>
                <body name="link2" pos="0 0 0.2">
                    <joint name="joint2" pos="0 0 -0.1" range="-90 90" axis="0 1 0"/>
                    <geom rgba=".2 .6 1 1"/>
                    <site name="end_effector" pos="0 0 0.1" size="0.01"/>
                </body>
            </body>
        </body>
    </worldbody>
</mujoco>"""
# load
model = mujoco.MjModel.from_xml_string(xml4)
data = mujoco.MjData(model)
mujoco.mj_resetDataKeyframe(model, data, 0)
viewer.launch(model,data)

### Example 3: Multi-body robot with control
We add actuators to the joints. The first joint is a torque controlled joint while the second joint is a position-joint. We set the inital pose to be $(0,\frac{\pi}{2})$ while the control inputs are zeros by default. So the simulation will show the process of the second joint rotating towards 0 degree and eventually settles down at zero. You can modify the damping kv term to see the response. Since we did not have any control over the first joint (i.e. desired torque is always zero), the first joint may rotate due to the second body's motion. We can add damping to the first joint. This can be done directly through the \<joint\/> element. 



In [9]:
import numpy as np
import mujoco
import mujoco.viewer as viewer

xml4="""<mujoco model="3R_robot">
    <compiler angle="degree"/>
    <asset>  
        <texture name="grid" type="2d" builtin="checker" rgb1=".1 .2 .3"
        rgb2=".2 .3 .4" width="300" height="300" mark="none"/>
        <material name="grid" texture="grid" texrepeat="6 6" texuniform="true" reflectance=".2"/>
    </asset>
    <default>
        <joint type="hinge" axis="0 0 1" limited="true"/>
        <geom type="cylinder" size=".025 .1" />
    </default>

    <worldbody>
        <light diffuse=".5 .5 .5" pos="0 0 3" dir="0 0 -1"/>
        <geom type="plane" size="1 1 0.1" material="grid"/>        
        <body name="BaseLink" pos="0 0 0.1">  
            <geom type="cylinder" pos="0 0 0" size=".025 .1" />  
            <body name="link1" pos="0 0.1 0.125" euler="-90 0 0">
                <joint name="joint1" pos="0 0 -0.1" range="-90 90" axis ="0 1 0" damping="0.1"/>
                <geom pos="0 0 0" rgba=".6 .2 .2 1"/>
                <body name="link2" pos="0 0 0.2">
                    <joint name="joint2" pos="0 0 -0.1" range="-90 90" axis="0 1 0"/>
                    <geom rgba=".2 .6 1 1"/>
                    <site name="end_effector" pos="0 0 0.1" size="0.01"/>
                </body>
            </body>
        </body>
    </worldbody>
    <!-- Actuators for controlling the joints -->
    <actuator>
        <motor name="torque_control" joint="joint1" ctrlrange="-10 10"/> 
        <position name="position_control" joint="joint2" kp="2" kv="0.02" ctrlrange="-90 90"/>
    </actuator>
    <!-- Sensors for monitoring joint positions and torques -->
    <sensor>
        <jointpos name="joint1_position_sensor" joint="joint1"/> <!-- Position sensor for joint1 -->
        <jointpos name="joint2_position_sensor" joint="joint2"/> <!-- Position sensor for joint2 -->
    </sensor>
    <keyframe>
        <!-- Initial configuration -->
        <key name="initial_pose" qpos="0 -1.57"/>        
        <!-- Bent configuration -->
        <key name="bent_pose" qpos="1.57079 0.78539"/>
    </keyframe>

</mujoco>"""

model = mujoco.MjModel.from_xml_string(xml4)
data = mujoco.MjData(model)
mujoco.mj_resetDataKeyframe(model, data, 1)
viewer.launch(model,data)

In [10]:
#height map example

import numpy as np
from PIL import Image
import time
import mujoco
import mujoco.viewer

height_map = np.random.rand(100, 100) * 255  
height_map = height_map.astype(np.uint8)

Image.fromarray(height_map).save("terrain.png")


xml_string = """
<mujoco model="height_field_terrain">
       <asset>
        <hfield name="terrain_heightfield" file="terrain.png" size="50 50 1 1" />
    </asset>

    <worldbody>
        <light directional="true" diffuse=".8 .8 .8" pos="0 0 10" dir="0 0 -10"/>
        <geom name="terrain_hfield" type="hfield" hfield="terrain_heightfield" pos="0 0 0" size="5 5 1"/>
        <geom name="terrain_box" type="box" pos="0 0 2" size="1 1 1" rgba="1 0 0 1"/>
        <geom name="ground" type="plane" pos="5 0 3" size="2 2 0.1" rgba="0 0 1 1"/>
    </worldbody>
</mujoco>
"""

m = mujoco.MjModel.from_xml_string(xml_string)
d = mujoco.MjData(m)
viewer.launch(m,d)